# Śledzenie rozgyrwki bilarda

## Import niezbędnych bibliotek

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import cv2
from collections import deque
# import PIL
# import os
# import sys
# import glob
# import random

# from pprint import pprint
from ipywidgets import Video

# from PIL import Image
# from PIL.ExifTags import TAGS

## Analiza wideo

### Wczytanie wideo i jego podstawowych właściwości

In [2]:
video = cv2.VideoCapture('data/9-ball/video11.mov')
if video.isOpened():
    print('Movie loaded :)')

width = int(video.get(3))
height = int(video.get(4))
print(f'Wysokość i szerokość obrazu: {height} x {width}')

fps = video.get(cv2.CAP_PROP_FPS)
print(f'Liczba klatek na sekundę: {fps}')

Movie loaded :)
Wysokość i szerokość obrazu: 1116 x 2038
Liczba klatek na sekundę: 56.6412213740458


### Wyświetlenie analizowanego wideo

In [3]:
Video.from_file('data/9-ball/video11.mov')

Video(value=b'\x00\x00\x00\x14ftypqt  \x00\x00\x00\x00qt  \x00\x00\x00\x08wide\x00D\xd0mmdat\x00\x00\x00\x1e\x…

### Oddzielenie tła od ruszających się obiektów

In [4]:
video_foreground_knn = cv2.VideoWriter('foreground_knn.avi', cv2.VideoWriter_fourcc(*'DIVX'), fps, (width, height), 0)
video_foreground_mog2 = cv2.VideoWriter('foreground_mog2.avi', cv2.VideoWriter_fourcc(*'DIVX'), fps, (width, height), 0)

foreground_knn = cv2.createBackgroundSubtractorKNN()
foreground_mog2 = cv2.createBackgroundSubtractorMOG2()

video.set(cv2.CAP_PROP_POS_FRAMES, 0)
while video.isOpened():
  ret, frame = video.read()

  if ret:
    video_foreground_knn.write(foreground_knn.apply(frame))
    video_foreground_mog2.write(foreground_mog2.apply(frame))
  else:
    break

video_foreground_knn.release()
video_foreground_mog2.release()

### Konwersja formatów wygenerowanych wideo

In [5]:
!ffmpeg -hide_banner -loglevel error -i foreground_knn.avi -y foreground_knn.mp4
!ffmpeg -hide_banner -loglevel error -i foreground_mog2.avi -y foreground_mog2.mp4

### Wyświetlenie wyników działania algorytmów oddzielających tło od ruszających się obiektów (maskujących tło)

In [6]:
print('Algorytm cv2.createBackgroundSubtractorKNN()')
Video.from_file('foreground_knn.mp4')

Algorytm cv2.createBackgroundSubtractorKNN()


Video(value=b'\x00\x00\x00 ftypisom\x00\x00\x02\x00isomiso2avc1mp41\x00\x00\x00\x08free\x00\r\xddimdat\x00\x00…

In [7]:
print('Algorytm cv2.createBackgroundSubtractorMOG2()')
Video.from_file('foreground_mog2.mp4')

Algorytm cv2.createBackgroundSubtractorMOG2()


Video(value=b'\x00\x00\x00 ftypisom\x00\x00\x02\x00isomiso2avc1mp41\x00\x00\x00\x08free\x00\x12\xb16mdat\x00\x…

Po analizie wyników działania powyższych algorytmów do dalszej pracy wybieram algorytm KNN, który osiągnął lepszy wynik od MOG2.

### Poszukiwanie współrzędnych bil na filmie

In [1]:
video = cv2.VideoCapture('data/9-ball/video9.mov')

while True:
    # Take each frame
    _, frame = video.read()

    # If video ends then break
    if frame is None:
        break

    blur = cv2.medianBlur(frame,5)
    cimg = cv2.cvtColor(blur,cv2.COLOR_BGR2GRAY)
    circles = cv2.HoughCircles(cimg,cv2.HOUGH_GRADIENT,1,10,
                                param1=50,param2=30,minRadius=10,maxRadius=30)

    circles = np.uint16(np.around(circles))

    for i in circles[0,:]:
        # draw the outer circle
        cv2.circle(frame,(i[0],i[1]),i[2],(0,255,0),2)
        # draw the center of the circle
        cv2.circle(frame,(i[0],i[1]),2,(0,0,255),3)

    cv2.imshow('frame',frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()

NameError: name 'cv2' is not defined